In [1]:
\l E:\pycharm\PycharmProjects\kdbq\columbiaHdb\ / Loading and Checking data
\c 50 10000

In [2]:
select from bin10 where date=2019.01.03

date       time     id trade     mid      spread        vol         adv          
---------------------------------------------------------------------------------
2019.01.03 09:30:00 0  454071.8  91.715   0.001148717   0.000446367 1.122415e+008
2019.01.03 09:30:10 0  -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008
2019.01.03 09:30:20 0  -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008
2019.01.03 09:30:30 0  -78884.65 91.65646 0.000612649   0.000446367 1.122415e+008
2019.01.03 09:30:40 0  -24705.54 91.60259 0.0008423923  0.000446367 1.122415e+008
2019.01.03 09:30:50 0  -92166.19 91.58385 0.0005871219  0.000446367 1.122415e+008
2019.01.03 09:31:00 0  -22823.01 91.56276 0.000663703   0.000446367 1.122415e+008
2019.01.03 09:31:10 0  -4940.182 91.59088 0.0005615949  0.000446367 1.122415e+008
2019.01.03 09:31:20 0  -41691.88 91.59322 0.0005360678  0.000446367 1.122415e+008
2019.01.03 09:31:30 0  -539826.4 91.60727 0.0007402842  0.000446367 1.122415e+008
2019.01.03 09:31

# Exercise 1

## For each correlation, create a synthetic alpha. The result should be in long format (duplicate each table row for each alpha).

In [3]:
dts:select distinct date from bin10
dts_list: exec date from dts /distinct date
deltas0:{first[x] -': x }
u12: {[n] -6f+sum n cut (12* n)?1f} / standard normal distribution generator
rhos:0 0.05 0.3 / rho coefs
b_float : rhos[2]* sqrt 1- xexp[rhos[2];2] /fixed b coef
sys_alpha:{ [dt;rhos]
    tb1:select from bin10 where date=dt;
    tb1: update reverse fills reverse date,
         reverse fills reverse mid,
         reverse fills reverse spread,
         reverse fills reverse vol,
         reverse fills reverse adv
         by id from tb1;
    tb1: update Rt: -1+(last mid)%mid by id from tb1;
    tb1_1 : update sigma_R: sqrt (6*(var deltas0[Rt])) by id from tb1; / time 6 to get minute frequency volatility
    tb1_2: update wt: sigma_R*sums (0,u12[-1+count time]) %sqrt 6 by id from tb1_1; / %6 to get minute frequency standard normal distribution
    /duplicate table rows by rho
    tb2: tb1_2 cross ([]rho:rhos;strat:0 1 2);
    update at:(Rt*rho*b_float%sqrt 1- xexp[rho;2])+b_float*wt- last wt by id,rho from tb2
    }

tb1:sys_alpha[2019.01.03;rhos]
select date,time,id,rho,strat,at from tb1

date       time     id rho  strat at          
----------------------------------------------
2019.01.03 09:30:00 0  0    0     -0.002199551
2019.01.03 09:30:00 0  0.05 1     -0.002623439
2019.01.03 09:30:00 0  0.3  2     -0.004862345
2019.01.03 09:30:10 0  0    0     -0.002289415
2019.01.03 09:30:10 0  0.05 1     -0.002715076
2019.01.03 09:30:10 0  0.3  2     -0.004963354
2019.01.03 09:30:20 0  0    0     -0.002336944
2019.01.03 09:30:20 0  0.05 1     -0.002741634
2019.01.03 09:30:20 0  0.3  2     -0.004879142
2019.01.03 09:30:30 0  0    0     -0.002399362
2019.01.03 09:30:30 0  0.05 1     -0.00281437 
2019.01.03 09:30:30 0  0.3  2     -0.005006375
2019.01.03 09:30:40 0  0    0     -0.002178551
2019.01.03 09:30:40 0  0.05 1     -0.002585377
2019.01.03 09:30:40 0  0.3  2     -0.004734168
2019.01.03 09:30:50 0  0    0     -0.002258273
2019.01.03 09:30:50 0  0.05 1     -0.002662251
2019.01.03 09:30:50 0  0.3  2     -0.004795996
2019.01.03 09:31:00 0  0    0     -0.002361504
2019.01.03 09

## For each synthetic alpha, backtest the optimal trading strategy. The result should be in long format (duplicate each table row for each strategy).

In [4]:
beta_float : log[2]%60
opt_strat:{[tb]
    /generate dleta alpha and impact
    tb: update diff_at: 0^(at-xprev[60;at])%10 by id,rho from tb;
    tb: update It: 0.5* at-diff_at%beta_float by id,rho from tb;
    tb:update I_ : exp[neg beta_float%6]*0^xprev[1;It] by id,rho from tb;
    update dq:(It-I_)%(8*vol*sqrt (6*60*6.5))%adv by id,rho from tb
    }
tb1_1:opt_strat[tb1]
select date,time,id,rho,strat,at,It,I_,dq from tb1_1

date       time     id rho  strat at           It           I_           dq       
----------------------------------------------------------------------------------
2019.01.03 09:30:00 0  0    0     -0.002199551 -0.001099776 0            -714607.5
2019.01.03 09:30:00 0  0.05 1     -0.002623439 -0.001311719 0            -852323.4
2019.01.03 09:30:00 0  0.3  2     -0.004862345 -0.002431172 0            -1579717 
2019.01.03 09:30:10 0  0    0     -0.002289415 -0.001144707 -0.00109766  -30570.04
2019.01.03 09:30:10 0  0.05 1     -0.002715076 -0.001357538 -0.001309196 -31411.38
2019.01.03 09:30:10 0  0.3  2     -0.004963354 -0.002481677 -0.002426496 -35855.18
2019.01.03 09:30:20 0  0    0     -0.002336944 -0.001168472 -0.001142505 -16872.55
2019.01.03 09:30:20 0  0.05 1     -0.002741634 -0.001370817 -0.001354927 -10325.09
2019.01.03 09:30:20 0  0.3  2     -0.004879142 -0.002439571 -0.002476903 24257.57 
2019.01.03 09:30:30 0  0    0     -0.002399362 -0.001199681 -0.001166225 -21739.18
2019

## AB test

In [5]:
abTest :{[ tbl ; strat1 ; strat2 ; prob1 ]
    /process
    /count stock,day pair amount as ct
    /generate ct random numbers, from 1 to ct without replacement for each date,stock pair
    /decide threshold =  int(prob*ct)
    /for rows with number smaller than threshold we assign it with strat1 otherwise strat2
    / left join the above table to the original table

    /stock day pair tbl
    pair_tb: select rand_num:0 by date,id from tbl;
    ct: count pair_tb;
    threshold: "i"$prob1*ct; / get threshold to assign strats
    pair_tb: update rand_num: (neg ct)?til ct from pair_tb;
    pair_tb: update ab_strat: 1h$rand_num<threshold from pair_tb; / asssign boolean
    pair_tb: select date,id,ab_strat: ab_strat*strat1+(1-ab_strat)*strat2 from pair_tb;
    tbl:tbl lj xkey[`date`id;pair_tb] /left join tabls
    }
tb1_2: abTest[tb1_1;1;0;0.8]
select date,time,id,strat,ab_strat,at,It,I_,dq from tb1_2

date       time     id strat ab_strat at           It           I_           dq       
--------------------------------------------------------------------------------------
2019.01.03 09:30:00 0  0     1        -0.002199551 -0.001099776 0            -714607.5
2019.01.03 09:30:00 0  1     1        -0.002623439 -0.001311719 0            -852323.4
2019.01.03 09:30:00 0  2     1        -0.004862345 -0.002431172 0            -1579717 
2019.01.03 09:30:10 0  0     1        -0.002289415 -0.001144707 -0.00109766  -30570.04
2019.01.03 09:30:10 0  1     1        -0.002715076 -0.001357538 -0.001309196 -31411.38
2019.01.03 09:30:10 0  2     1        -0.004963354 -0.002481677 -0.002426496 -35855.18
2019.01.03 09:30:20 0  0     1        -0.002336944 -0.001168472 -0.001142505 -16872.55
2019.01.03 09:30:20 0  1     1        -0.002741634 -0.001370817 -0.001354927 -10325.09
2019.01.03 09:30:20 0  2     1        -0.004879142 -0.002439571 -0.002476903 24257.57 
2019.01.03 09:30:30 0  0     1        -0.00

## Implement an abTest that randomly assigns each (stock, day) pair to one of two strategies.

In [6]:
abTest_compact:{[dt;strat1;strat2;prob1]
    tb1:sys_alpha[dt;rhos]; /generate alpha
    tb1:abTest[tb1;strat1;strat2;prob1]; /generate random strat
    opt_strat[tb1] /optimal strategy
    }


## For each day, bucket all stocks into three equal-size groups: low, medium, and high volatility. Implement the version of abTest that uses stratified sampling and randomizes within each volatility bucket. That is, for each day and within each volatility bucket, we require that the number of stocks that use strat1 is precisely (number-ofstock * prob1), up to rounding to the nearest integer.

In [7]:
vol_bucket:{[dt ; strat1 ; strat2 ; prob1]
    tb1:select from bin10 where date=dt;
    tb1: update reverse fills reverse date,
         reverse fills reverse mid,
         reverse fills reverse spread,
         reverse fills reverse vol,
         reverse fills reverse adv
         by id from tb1;
    /sum all the vol by id to get daily vol for each stock each day
    vol_tb: select vol_daily:sum vol by date,id from tb1;
    /assign bucket
    vol_tb: update bucket: 3 xrank vol_daily from vol_tb;
    /count amount and amount for each strat
    vol_tb:update ct: count id,thres: "i"$prob1*count id by bucket from vol_tb;
    vol_tb: update rand_num: (neg first ct)?til first ct by bucket from vol_tb;
    / asssign boolean
     vol_tb: update ab_strat: 1h$rand_num<first thres by bucket from vol_tb;
     select date,id,bucket,ab_strat: ab_strat*strat1+(1-ab_strat)*strat2 from vol_tb
    }
bucket_tb:vol_bucket[2019.01.03;1;0;0.5]
select ct: count id by date,bucket,ab_strat from bucket_tb

date       bucket ab_strat| ct
--------------------------| --
2019.01.03 0      0       | 83
2019.01.03 0      1       | 84
2019.01.03 1      0       | 83
2019.01.03 1      1       | 84
2019.01.03 2      0       | 83
2019.01.03 2      1       | 84


# Exercise 2
The baseline scenario is strat1 follows the ρ1 alpha signal and strat2 does not trade.

## Simulate an AB-test with prob1 at 80%. What is the average daily P&L of this randomized strategy? What is the average daily P&L for each strategy?

Discritized PNL can be written as:

$Y = \Sigma^{N-1}_{m=0} (\Delta_mQ-Q_{m-1}R^m_{m-1})R^N_m -(I_{m-1}-\frac{1}{2}\Delta_mI)\Delta_mQ$

In [8]:
/compute pnl by each date and raze them later to avoid overflow
pnl_comp:{ [dt;strat1;strat2;prob1]
    ab_tb: abTest_compact[dt;strat1;strat2;prob1]; /table with ab_test strategy
    pnl_tb:update Q:mid*sums dq,R_adj:0^(mid%xprev[1;mid])-1 by date,strat,id from ab_tb; /compute daily pnl for each date,strategy,id
    /unkeyed
    0!select first ab_strat,pnl_per :sum ((0^(deltas0[Q]-xprev[1;Q]*R_adj)*Rt) - (0.5*(I_+It)*deltas0[Q]))% first adv,alpha_str:first at by date,strat,id from pnl_tb
    }

In [9]:
\t pnl_tb: raze pnl_comp[;1;0;0.8] peach 40#dts_list
pnl_tb

94816


date       strat id ab_strat pnl_per      alpha_str    
-------------------------------------------------------
2019.01.02 0     0  0        -0.05829471  -0.008692722 
2019.01.02 0     1  1        -0.0642954   -0.001712132 
2019.01.02 0     2  1        -0.02710849  6.728044e-005
2019.01.02 0     3  1        -0.04223786  0.002885922  
2019.01.02 0     4  0        -0.01833708  0.01241003   
2019.01.02 0     5  1        -0.05431523  0.004170791  
2019.01.02 0     6  1        0.009732774  -0.002015065 
2019.01.02 0     7  0        -0.1661224   -0.004062989 
2019.01.02 0     8  1        -0.01997684  0.005071437  
2019.01.02 0     9  1        -0.1970703   -0.004835338 
2019.01.02 0     10 1        -0.02417324  0.007540943  
2019.01.02 0     11 1        -0.009395231 -0.007827565 
2019.01.02 0     12 0        -0.1612796   0.004638413  
2019.01.02 0     13 1        -0.03614624  -0.009853591 
2019.01.02 0     14 1        -0.02255106  -0.004382287 
2019.01.02 0     15 1        -0.0409588   0.0047

Average Daily PNL

In [10]:
/ for randomized ab-test strat we have strat=ab_strat
daily_avg_pnl_tb: select ab_avg_pnl_per:avg pnl_per from pnl_tb where strat=ab_strat
daily_avg_pnl_tb

ab_avg_pnl_per
--------------
-0.04545292   


Average daily PNL for each strategy

In [11]:
pnl_avg_strat_tb : select avg_pnl_per: avg pnl_per by strat from pnl_tb
pnl_avg_strat_tb

strat| avg_pnl_per
-----| -----------
0    | -0.04597419
1    | -0.04532189
2    | -0.04454173


## For every day, compute the daily t-stat (mean/sdev) of each strategy’s P&L across all stocks. Use suitably normalized units.

In [12]:
/daily t_statistics for each date and strategy pair
t_stat_tb: select t_stat :0^( (avg pnl_per) % dev pnl_per) by date,strat from pnl_tb
t_stat_tb

date       strat| t_stat    
----------------| ----------
2019.01.02 0    | -0.4297323
2019.01.02 1    | -0.4329327
2019.01.02 2    | -0.4613778
2019.01.03 0    | -0.2696597
2019.01.03 1    | -0.2722508
2019.01.03 2    | -0.3006424
2019.01.04 0    | -0.4344475
2019.01.04 1    | -0.433304 
2019.01.04 2    | -0.4305586
2019.01.07 0    | -0.3774111
2019.01.07 1    | -0.3778625
2019.01.07 2    | -0.3874849
2019.01.08 0    | -0.3189941
2019.01.08 1    | -0.3213077
2019.01.08 2    | -0.3396934
2019.01.09 0    | 0.02409518
2019.01.09 1    | 0.02732186
2019.01.09 2    | 0.04932743
2019.01.10 0    | -0.4065995
2019.01.10 1    | -0.4106799
2019.01.10 2    | -0.4326024
2019.01.11 0    | -0.5099572
2019.01.11 1    | -0.5148969
2019.01.11 2    | -0.5422928
2019.01.14 0    | -0.4985066
2019.01.14 1    | -0.5001265
2019.01.14 2    | -0.5081768
2019.01.15 0    | -0.4987466
2019.01.15 1    | -0.4985574
2019.01.15 2    | -0.493357 
2019.01.16 0    | -0.2705608
2019.01.16 1    | -0.2736917
2019.01.16 2  

##  For every month, compute the t-stat (mean/sdev) of each strategy’s P&L across all (stock,days). Use suitably normalized units

In [13]:
m_ab_tb: update month:date.month from pnl_tb /generate month column
m_tstat_tb: select tstat:0^( (avg pnl_per) % dev pnl_per) by month,strat from m_ab_tb /monthly t-statisitcs
m_tstat_tb

month   strat| tstat     
-------------| ----------
2019.01 0    | -0.3624109
2019.01 1    | -0.3618289
2019.01 2    | -0.3595641
2019.02 0    | -0.3635305
2019.02 1    | -0.3640693
2019.02 2    | -0.3681802


## Repeat Questions 1-3 for prob1 = 0.1, 0.2, ..., 0.9. Comment on the P&L and uncertainty trade-off across A-B allocations.

Besides repeating above steps, I calculate each strategies' tstat across all dates within duration and across all stocks. Basing on these statistiscs I conclude whether these strategies are significantly different from each other.

In [17]:
p4_proc:{[prob;strats]
    /show info
    show string `pairs;
    show strats;
    show string `prob;
    show prob;
    /pnl by each date,strat,id
    pnl_tb: raze pnl_comp[;strats[0];strats[1];prob] peach 10#dts_list;
    pnl_ab_tb: select from pnl_tb where strat=ab_strat;
    /add randomized strat as new strat with index 3
    pnl_ab_tb: update strat:3 from pnl_ab_tb;
    pnl_tb: pnl_ab_tb,pnl_tb;
    pnl_tb: xasc[`date`strat`id;pnl_tb];
    strat_tb : 0!select strat_pnl_per: avg pnl_per by date,strat from pnl_tb;
    show string `strats_pnl_percent;
    show strat_tb;
    /daily strategy tstat
    tstat_tb: select tstat :0^( (avg pnl_per) % dev pnl_per) by date,strat from pnl_tb;
    show string `daily_tstat;
    show tstat_tb;
    /monthly strat tstat
    m_ab_tb: update month:date.month from pnl_tb; /generate month column
    m_tstat_tb: select tstat:0^( (avg pnl_per) % dev pnl_per) by month,strat from m_ab_tb;
    show string `monthly_tstat;
    show m_tstat_tb;
    /tstat for strategy across stocks and dates
    show `tstats_for_strategies;
    show select tstat :0^( (avg pnl_per) % dev pnl_per) by strat from pnl_tb;
    }

In [14]:
probs: 0.1+(til 9)%10
p4:raze p4_proc[;1,0] each probs

"pairs"
1 0
"prob"
0.1
"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.07171116  
2019.01.02 1     -0.07054903  
2019.01.02 2     -0.06891635  
2019.01.02 3     -0.07167413  
2019.01.03 0     -0.05912989  
2019.01.03 1     -0.058409    
2019.01.03 2     -0.05894673  
2019.01.03 3     -0.05910132  
2019.01.04 0     -0.05721924  
2019.01.04 1     -0.05646728  
2019.01.04 2     -0.05621731  
2019.01.04 3     -0.05710647  
2019.01.07 0     -0.05625669  
2019.01.07 1     -0.05645193  
2019.01.07 2     -0.06073117  
2019.01.07 3     -0.05615883  
2019.01.08 0     -0.05645536  
2019.01.08 1     -0.05651217  
2019.01.08 2     -0.05992497  
2019.01.08 3     -0.05651776  
2019.01.09 0     0.0001018835 
2019.01.09 1     0.0001077241 
2019.01.09 2     0.0001493297 
2019.01.09 3     0.00010267   
2019.01.10 0     -0.05190071  
2019.01.10 1     -0.05098904  
2019.01.10 2     -0.04932547  
2019.01.10 3     -0.05189237  
2019.01.11 0     -0.051589

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06841478  
2019.01.02 1     -0.06728933  
2019.01.02 2     -0.06585037  
2019.01.02 3     -0.06817164  
2019.01.03 0     -0.07854914  
2019.01.03 1     -0.07767126  
2019.01.03 2     -0.07737981  
2019.01.03 3     -0.07794092  
2019.01.04 0     -0.065248    
2019.01.04 1     -0.0645696   
2019.01.04 2     -0.06470818  
2019.01.04 3     -0.0650469   
2019.01.07 0     -0.05315048  
2019.01.07 1     -0.05352851  
2019.01.07 2     -0.05877319  
2019.01.07 3     -0.05328332  
2019.01.08 0     -0.06004354  
2019.01.08 1     -0.06009105  
2019.01.08 2     -0.06345468  
2019.01.08 3     -0.05998906  
2019.01.09 0     9.514532e-005
2019.01.09 1     0.0001015476 
2019.01.09 2     0.0001461203 
2019.01.09 3     9.417803e-005
2019.01.10 0     -0.05233658  
2019.01.10 1     -0.05136957  
2019.01.10 2     -0.04941366  
2019.01.10 3     -0.05191211  
2019.01.11 0     -0.04961042  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06968025  
2019.01.02 1     -0.06854611  
2019.01.02 2     -0.06706129  
2019.01.02 3     -0.06897358  
2019.01.03 0     -0.06271228  
2019.01.03 1     -0.06198743  
2019.01.03 2     -0.06250426  
2019.01.03 3     -0.06221886  
2019.01.04 0     -0.06677342  
2019.01.04 1     -0.06604032  
2019.01.04 2     -0.06588996  
2019.01.04 3     -0.06642637  
2019.01.07 0     -0.05859416  
2019.01.07 1     -0.05882704  
2019.01.07 2     -0.06330508  
2019.01.07 3     -0.05885581  
2019.01.08 0     -0.06701297  
2019.01.08 1     -0.0669998   
2019.01.08 2     -0.07004296  
2019.01.08 3     -0.06695975  
2019.01.09 0     2.565492e-005
2019.01.09 1     3.252228e-005
2019.01.09 2     7.955117e-005
2019.01.09 3     2.755886e-005
2019.01.10 0     -0.05723264  
2019.01.10 1     -0.05624725  
2019.01.10 2     -0.05419424  
2019.01.10 3     -0.05650651  
2019.01.11 0     -0.04283545  
2019.01.11 1     -

Tstats corresponding to the randomized strategy is quite small, around -0.3 and far from +-1.96, which shows no significant difference between the two strategies. Therefore, we cannot tell difference between the two strategies.

## Repeat Questions 1-4 for every strategy pair. Comment on the amount of A-B testing needed based on an alpha’s strength and the trader’s waiting time.

In [19]:
pairs:((0;1);(0;2);(1;2))
/repeat 1-4 process for each pair
proc_5_1:{ [strats]
    p4_proc[;strats] each probs;
    }
show pairs
proc_5_1 each pairs

0 1
0 2
1 2
"pairs"
0 1
"prob"
0.1
"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.07280702  
2019.01.02 1     -0.07165393  
2019.01.02 2     -0.07006902  
2019.01.02 3     -0.07280702  
2019.01.03 0     -0.07451495  
2019.01.03 1     -0.07367269  
2019.01.03 2     -0.07356933  
2019.01.03 3     -0.07451495  
2019.01.04 0     -0.06905069  
2019.01.04 1     -0.06822095  
2019.01.04 2     -0.06756018  
2019.01.04 3     -0.06905069  
2019.01.07 0     -0.05774164  
2019.01.07 1     -0.05809199  
2019.01.07 2     -0.06319044  
2019.01.07 3     -0.05774164  
2019.01.08 0     -0.05540079  
2019.01.08 1     -0.05544299  
2019.01.08 2     -0.05877858  
2019.01.08 3     -0.05540079  
2019.01.09 0     0.0001365568 
2019.01.09 1     0.0001420724 
2019.01.09 2     0.0001819613 
2019.01.09 3     0.0001365568 
2019.01.10 0     -0.05298171  
2019.01.10 1     -0.05204216  
2019.01.10 2     -0.05023127  
2019.01.10 3     -0.05298171  
2019.01.11 0  

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06480153  
2019.01.02 1     -0.06393342  
2019.01.02 2     -0.06385373  
2019.01.02 3     -0.06480153  
2019.01.03 0     -0.07340706  
2019.01.03 1     -0.0725994   
2019.01.03 2     -0.07267887  
2019.01.03 3     -0.07340706  
2019.01.04 0     -0.06083678  
2019.01.04 1     -0.06020579  
2019.01.04 2     -0.06059481  
2019.01.04 3     -0.06083678  
2019.01.07 0     -0.06111796  
2019.01.07 1     -0.06137682  
2019.01.07 2     -0.06599204  
2019.01.07 3     -0.06111796  
2019.01.08 0     -0.05995512  
2019.01.08 1     -0.05994858  
2019.01.08 2     -0.06302672  
2019.01.08 3     -0.05995512  
2019.01.09 0     0.0001011505 
2019.01.09 1     0.0001070056 
2019.01.09 2     0.000148688  
2019.01.09 3     0.0001011505 
2019.01.10 0     -0.06401785  
2019.01.10 1     -0.06291174  
2019.01.10 2     -0.06022116  
2019.01.10 3     -0.06401785  
2019.01.11 0     -0.04379935  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06284327  
2019.01.02 1     -0.06189172  
2019.01.02 2     -0.0613713   
2019.01.02 3     -0.06284327  
2019.01.03 0     -0.06648156  
2019.01.03 1     -0.0658021   
2019.01.03 2     -0.06655867  
2019.01.03 3     -0.06648156  
2019.01.04 0     -0.05624913  
2019.01.04 1     -0.05552574  
2019.01.04 2     -0.05542673  
2019.01.04 3     -0.05624913  
2019.01.07 0     -0.05803498  
2019.01.07 1     -0.05819759  
2019.01.07 2     -0.06230443  
2019.01.07 3     -0.05803498  
2019.01.08 0     -0.05996086  
2019.01.08 1     -0.06004083  
2019.01.08 2     -0.06357593  
2019.01.08 3     -0.05996086  
2019.01.09 0     1.174528e-005
2019.01.09 1     1.93479e-005 
2019.01.09 2     7.02603e-005 
2019.01.09 3     1.174528e-005
2019.01.10 0     -0.05356034  
2019.01.10 1     -0.05248881  
2019.01.10 2     -0.04998082  
2019.01.10 3     -0.05356034  
2019.01.11 0     -0.04056174  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06045461  
2019.01.02 1     -0.05933053  
2019.01.02 2     -0.05789882  
2019.01.02 3     -0.06045461  
2019.01.03 0     -0.06794004  
2019.01.03 1     -0.06715181  
2019.01.03 2     -0.06733387  
2019.01.03 3     -0.06794004  
2019.01.04 0     -0.06632443  
2019.01.04 1     -0.06562542  
2019.01.04 2     -0.06565511  
2019.01.04 3     -0.06632443  
2019.01.07 0     -0.05597065  
2019.01.07 1     -0.0561966   
2019.01.07 2     -0.06063804  
2019.01.07 3     -0.05597065  
2019.01.08 0     -0.06099338  
2019.01.08 1     -0.06102596  
2019.01.08 2     -0.06431075  
2019.01.08 3     -0.06099338  
2019.01.09 0     0.0003134923 
2019.01.09 1     0.0003184295 
2019.01.09 2     0.0003552637 
2019.01.09 3     0.0003134923 
2019.01.10 0     -0.04699343  
2019.01.10 1     -0.0461134   
2019.01.10 2     -0.04461693  
2019.01.10 3     -0.04699343  
2019.01.11 0     -0.04711741  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06046505  
2019.01.02 1     -0.05951558  
2019.01.02 2     -0.05900612  
2019.01.02 3     -0.06046505  
2019.01.03 0     -0.06507719  
2019.01.03 1     -0.06431621  
2019.01.03 2     -0.06464223  
2019.01.03 3     -0.06507719  
2019.01.04 0     -0.06622434  
2019.01.04 1     -0.06537814  
2019.01.04 2     -0.06463047  
2019.01.04 3     -0.06622434  
2019.01.07 0     -0.05778302  
2019.01.07 1     -0.05797658  
2019.01.07 2     -0.06224691  
2019.01.07 3     -0.05778302  
2019.01.08 0     -0.06010822  
2019.01.08 1     -0.06017783  
2019.01.08 2     -0.06365816  
2019.01.08 3     -0.06010822  
2019.01.09 0     0.0001356428 
2019.01.09 1     0.0001408717 
2019.01.09 2     0.0001792467 
2019.01.09 3     0.0001356428 
2019.01.10 0     -0.06238634  
2019.01.10 1     -0.0612955   
2019.01.10 2     -0.05868557  
2019.01.10 3     -0.06238634  
2019.01.11 0     -0.0477939   
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.05807956  
2019.01.02 1     -0.0570571   
2019.01.02 2     -0.05616217  
2019.01.02 3     -0.05807956  
2019.01.03 0     -0.04803722  
2019.01.03 1     -0.04761657  
2019.01.03 2     -0.04974016  
2019.01.03 3     -0.04803722  
2019.01.04 0     -0.06227787  
2019.01.04 1     -0.06157451  
2019.01.04 2     -0.06158124  
2019.01.04 3     -0.06227787  
2019.01.07 0     -0.05741637  
2019.01.07 1     -0.05765481  
2019.01.07 2     -0.06216225  
2019.01.07 3     -0.05741637  
2019.01.08 0     -0.05278172  
2019.01.08 1     -0.05281091  
2019.01.08 2     -0.05607779  
2019.01.08 3     -0.05278172  
2019.01.09 0     0.0001495757 
2019.01.09 1     0.0001546222 
2019.01.09 2     0.0001920337 
2019.01.09 3     0.0001495757 
2019.01.10 0     -0.0544868   
2019.01.10 1     -0.05351858  
2019.01.10 2     -0.05155631  
2019.01.10 3     -0.0544868   
2019.01.11 0     -0.04957375  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.07996194  
2019.01.02 1     -0.07883757  
2019.01.02 2     -0.07740433  
2019.01.02 3     -0.0796984   
2019.01.03 0     -0.07839507  
2019.01.03 1     -0.07740594  
2019.01.03 2     -0.07652689  
2019.01.03 3     -0.0783056   
2019.01.04 0     -0.06621687  
2019.01.04 1     -0.06546115  
2019.01.04 2     -0.06519139  
2019.01.04 3     -0.06604761  
2019.01.07 0     -0.05514087  
2019.01.07 1     -0.0553148   
2019.01.07 2     -0.05948146  
2019.01.07 3     -0.05521866  
2019.01.08 0     -0.06312504  
2019.01.08 1     -0.06320701  
2019.01.08 2     -0.06675267  
2019.01.08 3     -0.06311345  
2019.01.09 0     0.0001239502 
2019.01.09 1     0.0001303323 
2019.01.09 2     0.000174798  
2019.01.09 3     0.0001256214 
2019.01.10 0     -0.05680546  
2019.01.10 1     -0.05580247  
2019.01.10 2     -0.05365651  
2019.01.10 3     -0.05675625  
2019.01.11 0     -0.04522839  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06382674  
2019.01.02 1     -0.06284188  
2019.01.02 2     -0.0621455   
2019.01.02 3     -0.0637208   
2019.01.03 0     -0.06360027  
2019.01.03 1     -0.06286364  
2019.01.03 2     -0.06331826  
2019.01.03 3     -0.06337795  
2019.01.04 0     -0.06507236  
2019.01.04 1     -0.06426835  
2019.01.04 2     -0.06374353  
2019.01.04 3     -0.064726    
2019.01.07 0     -0.06054889  
2019.01.07 1     -0.06075956  
2019.01.07 2     -0.06512024  
2019.01.07 3     -0.06081109  
2019.01.08 0     -0.05930499  
2019.01.08 1     -0.05940123  
2019.01.08 2     -0.06302227  
2019.01.08 3     -0.05935217  
2019.01.09 0     0.0001054456 
2019.01.09 1     0.0001111194 
2019.01.09 2     0.0001518439 
2019.01.09 3     0.0001096192 
2019.01.10 0     -0.05523171  
2019.01.10 1     -0.05425492  
2019.01.10 2     -0.05224738  
2019.01.10 3     -0.05505394  
2019.01.11 0     -0.04519987  
2019.01.11 1     -

"strats_pnl_percent"
date       strat strat_pnl_per
------------------------------
2019.01.02 0     -0.06747272  
2019.01.02 1     -0.06651696  
2019.01.02 2     -0.06597435  
2019.01.02 3     -0.06678406  
2019.01.03 0     -0.06729031  
2019.01.03 1     -0.0665751   
2019.01.03 2     -0.06714283  
2019.01.03 3     -0.06685758  
2019.01.04 0     -0.05831638  
2019.01.04 1     -0.057618    
2019.01.04 2     -0.05765106  
2019.01.04 3     -0.05780976  
2019.01.07 0     -0.06283127  
2019.01.07 1     -0.06288587  
2019.01.07 2     -0.06642227  
2019.01.07 3     -0.0626415   
2019.01.08 0     -0.05309417  
2019.01.08 1     -0.05322129  
2019.01.08 2     -0.05700543  
2019.01.08 3     -0.05311037  
2019.01.09 0     0.0001848072 
2019.01.09 1     0.0001898903 
2019.01.09 2     0.000227495  
2019.01.09 3     0.0001903127 
2019.01.10 0     -0.05773438  
2019.01.10 1     -0.0566425   
2019.01.10 2     -0.05402704  
2019.01.10 3     -0.05691629  
2019.01.11 0     -0.04090593  
2019.01.11 1     -

::
::
::


In order to get waiting time given specific alpha strengths, we do experiments under the setting of pair (0,1) and prob 0.5 and compare the t-statisitcs of each strategy across dates and stocks for each duration.

In [16]:
duration: 10+10*til 18
p5_proc:{[prob;strats;dur]
    /show info
    show `experiment_duration;
    show dur;
    show string `pairs;
    show strats;
    show string `prob;
    show prob;
    /pnl by each date,strat,id
    pnl_tb: raze pnl_comp[;strats[0];strats[1];prob] peach dur#dts_list;
    pnl_ab_tb: select from pnl_tb where strat=ab_strat;
    /add randomized strat as new strat with index 3
    pnl_ab_tb: update strat:3 from pnl_ab_tb;
    pnl_tb: pnl_ab_tb,pnl_tb;
    pnl_tb: xasc[`date`strat`id;pnl_tb];
    /tstat for strategy
    show `tstats_for_strategies;
    dur_tb:select avg_alpha_strength: avg alpha_str,t_stat :0^( (avg pnl_per) % dev pnl_per) by strat from pnl_tb;
    show dur_tb;
    dur_tb
    }
dur_tbs:p5_proc[0.5;0 1;] each duration

`experiment_duration
10
"pairs"
0 1
"prob"
0.5
`tstats_for_strategies
strat| avg_alpha_strength t_stat    
-----| -----------------------------
0    | 6.747532e-005      -0.402763 
1    | 0.0001584119       -0.4041786
2    | 0.0006387245       -0.4153273
3    | 6.747532e-005      -0.402763 
`experiment_duration
20
"pairs"
0 1
"prob"
0.5
`tstats_for_strategies
strat| avg_alpha_strength t_stat    
-----| -----------------------------
0    | -3.822251e-005     -0.4042133
1    | 2.845322e-005      -0.4038134
2    | 0.0003806239       -0.3973268
3    | -3.822251e-005     -0.4042133
`experiment_duration
30
"pairs"
0 1
"prob"
0.5
`tstats_for_strategies
strat| avg_alpha_strength t_stat    
-----| -----------------------------
0    | 4.398909e-005      -0.3723066
1    | 0.0001051592       -0.3749744
2    | 0.0004282499       -0.3871505
3    | 4.398909e-005      -0.3723066
`experiment_duration
40
"pairs"
0 1
"prob"
0.5
`tstats_for_strategies
strat| avg_alpha_strength t_stat    
-----| ----------

After 90-days duration, the t-statisitcs of ab_test synthetic strategy becomes relatively stable around -0.37, which is still not signifcant.
Alpha strenghts seems to have no effects over the effective duration.